## Setting

In [ ]:
""" Train.ipynb """

# Colab Drive Mount: 결과물을 내 드라이브에 저장하거나 불러오기 위해 필요
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Drive에서 데이터셋 불러오기
!cp /content/drive/MyDrive/img_align_celeba.zip /content/ # 데이터 저장 위치를 작성

!unzip -q /content/drive/MyDrive/img_align_celeba.zip -d /content/data/ # 데이터 저장 위치 작성

data_dir = '/content/data/img_align_celeba'

## Import / Parameter

In [ ]:
!pip install torch torchvision

import os
import time
import random
import glob
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split, Subset, TensorDataset, ConcatDataset
from torchvision import transforms, datasets, utils
from torchvision.utils import make_grid, save_image
from tqdm import tqdm
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

dataset.ipynb 파일 내 코드를 사용하시면 됩니다.

In [ ]:
# Parameter
batch_size = 16
sample_size = 30000
seed = 42

In [ ]:
""" 데이터 처리 """
# Transform 정의
transform = transforms.Compose([
    transforms.CenterCrop(178),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# CelebA 데이터셋
class CelebADataset(Dataset):
  def __init__(self, img_dir, transform=None):
    self.img_paths = sorted(glob.glob(os.path.join(img_dir, "*.jpg")))
    self.transform = transform

  def __len__(self):
    return len(self.img_paths)

  def __getitem__(self, idx):
    image = Image.open(self.img_paths[idx]).convert("RGB")
    if self.transform:
      image = self.transform(image)
    return image

# Dataset 로딩
full_dataset = CelebADataset(data_dir, transform=transform)
full_indices = list(range(len(full_dataset)))

# 30000장 샘플링
sampled_count = min(sample_size, len(full_dataset))
random.seed(seed)
sampled_indices = random.sample(full_indices, sampled_count)
np.save('/content/drive/MyDrive/ProGAN/train_indices.npy', sampled_indices)

subset_dataset = Subset(full_dataset, sampled_indices)

# 로딩 속도 확인
load_loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
start = time.time()
for _ in tqdm(load_loader, desc="Loading 30000 images"): pass
print(f"Loaded {sampled_count} images in {(time.time()-start):.2f}s")

# Train/Val 분할
train_img = int(0.8 * sampled_count)
val_img = sampled_count - train_img
train_ds, val_ds = random_split(subset_dataset, [train_img, val_img])

# DataLoader 생성
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# 평가용 데이터셋에서 학습에 사용된 인덱스를 제외한 나머지를 저장 - evaluate에서 사용
unused_indices = list(set(full_indices) - set(sampled_indices))
np.save('/content/drive/MyDrive/ProGAN/unused_indices_for_eval.npy', unused_indices)

## Model

model.ipynb 파일 내 코드를 사용하시면 됩니다.

In [ ]:
# 채널 리스트 / steps
channel_list = [128, 128, 128, 128, 64]
steps = 4

In [ ]:
""" WSConv2d """
class WSConv2d(nn.Module):
  # 입력 channel 수, 출력 channel 수, kernel 크기, stride(이동폭), padding(경계 처리), scale 보정 계수
  def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, gain=2):
    super().__init__()

    # 기본 Conv2d layer 생성
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Scale 계산
    self.scale = (gain / (in_channels * kernel_size ** 2)) ** 0.5

    # Bias를 따로 저장, conv layer에서는 bias 제거
    self.bias = self.conv.bias
    self.conv.bias = None

    # He 초기화 기준 -> weight/bias 초기화
    # conv.weight: 정규 분포 샘플링
    # bias: 모두 0으로 초기화
    nn.init.normal_(self.conv.weight)
    nn.init.zeros_(self.bias)



  # 입력값 * scale -> weight scaling 효과
  def forward(self, x):

    # bias는 channel별로 reshape후 더함
    out = self.conv(x * self.scale) + self.bias.view(1, -1, 1, 1)
    return out

In [ ]:
""" PixelNorm """
class PixelNorm(nn.Module):
  def __init__(self):
    super().__init__()
    # eps -> 분모에 사용
    self.eps = 1e-8


  # 각 픽셀마다 벡터의 크기를 1로 정규화
  # sqrt(mean+eps)로 pixel별 norm 산출
  # x를 norm으로 나눠 픽셀 벡터 크기 = 1로 정규화
  def forward(self, x):
    return x / torch.sqrt(torch.mean(x**2, dim=1, keepdim=True) + self.eps)

In [ ]:
""" Up/Down Sampling """
class UpDownSampling(nn.Module):
  def __init__(self, size):
    super().__init__()

    # scale_factor: 배율
    self.size = size


  def forward(self, x):
    # 최근접 보간 - 해상도 전환 시 빠르고 단순한 연산 수행
    return F.interpolate(x, scale_factor=self.size, mode="nearest")

In [ ]:
""" Minibatchstd """
# Batch 내 통계량 추가 -> 서로 다른 샘플 간 변별력 상승
class MinibatchStd(nn.Module):
  def forward(self, x):
    bs, _, h, w = x.size()

    # Channel별 픽셀 표준편차 계산 -> 전체 평균 -> (bsx1xhxw) 크기로 복제
    std = torch.std(x, dim=0).mean().repeat(bs, 1, h, w)

    # 원본 feature map에 std channel을 추가 -> 미세한 차이 학습 가능
    return torch.cat([x, std], dim=1)

In [ ]:
""" GeneratorBlock """
# 해상도를 2배씩 늘려 특징을 점진적으로 확장
class GeneratorConvBlock(nn.Module):
  def __init__(self, step, scale_size):
    super().__init__()

    # 2x upsampling layer: 크기를 scale_size배로 증가
    self.up = UpDownSampling(scale_size)

    # 2번의 WSConv + LeakyReLU + PixelNorm 반복
    # 첫 번째 WSConv2d: 채널 수를 이전 단계 -> 현재 단계로 변환
    self.conv1 = WSConv2d(channel_list[step-1], channel_list[step])

    # 두 번째 WSConv2d: 같은 채널 수 유지하며 추가 특징 학습
    self.conv2 = WSConv2d(channel_list[step], channel_list[step])

    # LeakyReLU 적용 -> 학습 안정화
    self.lrelu = nn.LeakyReLU(0.2)

    # PixelNorm: 픽셀 단위 정규화를 통한 학습 안정화
    self.pn = PixelNorm()


  def forward(self, x):
    # Upsampling -> 해상도 x2
    x = self.up(x)

    # 첫 번째 Conv2 -> LeakyReLU -> PixelNorm
    x = self.lrelu(self.conv1(x))
    x = self.pn(x)

    # 두 번째 Conv2 -> LeakyReLU -> PixelNorm
    x = self.lrelu(self.conv2(x))
    x = self.pn(x)

    return x

""" Generator Structure """
class Generator(nn.Module):
  def __init__(self, steps):
    super().__init__()
    self.steps = steps

    # --- 초기 블록: 4x4 ---
    # PixelNorm: 입력 z 정규화
    # ConvTranspose2d: 채널 list[0] -> 채널_list[0], kernel 4x4 -> 4x4 map 생성
    # LeakyReLU -> WSConv2d -> LeakyReLU -> PixelNorm
    self.init = nn.Sequential(
        PixelNorm(),
        nn.ConvTranspose2d(channel_list[0], channel_list[0], 4, 1, 0),
        nn.LeakyReLU(0.2),
        WSConv2d(channel_list[0], channel_list[0]),
        nn.LeakyReLU(0.2),
        PixelNorm()
    )

    # --- Progressive Block ---
    # init 블록 뒤 -> step=1부터 steps까지 GeneratorConvBlock 쌓기
    # 각 block이 해상도를 2배씩 증가시키며 특징 확장
    self.prog_blocks = nn.ModuleList([self.init] + [GeneratorConvBlock(step, 2) for step in range(1, steps+1)])

    # --- toRGB layer ---
    # 마지막 feature map을 RGB 이미지로 변환 & kernel 크기 1x1로 channel 변환만 수행
    self.toRGB = WSConv2d(channel_list[steps], 3, kernel_size=1, stride=1, padding=0)


  def forward(self, z):
    out = z

    # 초기 4x4 block
    out = self.prog_blocks[0](out)

    # 점진적 해상도 증가 블록 순차 적용(해상도 x2 -> feature map 확장)
    for block in self.prog_blocks[1:]:
      out = block(out)

    # toRGB: 마지막 feature map을 RGB 이미지로 변환
    return self.toRGB(out)

In [ ]:
""" DiscriminatorBlock """
# 해상도를 단계별로 줄이며 특징 추출
class DiscriminatorConvBlock(nn.Module):
    def __init__(self, step):
      super().__init__()

      # ---각 해상도별 fromRGB layer와 block을 역순으로 쌓음---
      # 첫 번째 WSConv2d: 같은 채널 수 유지하며 nonlinear activation 전 특징 추출
      self.conv1 = WSConv2d(channel_list[step], channel_list[step])

      # 두 번째 WSConv2d: 이전 단계 채널 수로 줄이면서 세밀한 특징 학습
      self.conv2 = WSConv2d(channel_list[step], channel_list[step-1])

      # AvgPool: 해상도 절반으로 줄이는 downsampling
      self.down = nn.AvgPool2d(2,2)

      # LeakyReLU 적용 -> 학습 안정화
      self.lrelu = nn.LeakyReLU(0.2)

    def forward(self,x):
      # conv1 -> LeakyReLU
      x = self.lrelu(self.conv1(x))

      # conv2 -> LeakyReLU
      x = self.lrelu(self.conv2(x))

      # 해상도 절반으로 축소
      return self.down(x)

""" Discriminator Structure """
class Discriminator(nn.Module):
    def __init__(self, steps):
      super().__init__()
      self.steps=steps

      # 각 해상도별 fromRGB layer를 list 순서대로 저장
      self.fromrgb_layers = nn.ModuleList()

      # 단계별 Conv block을 순서대로 저장
      self.prog_blocks = nn.ModuleList()

      # 높은 해상도 -> 낮은 해상도 순으로 layer 구성
      for s in range(steps, 0, -1):

        # RGB 이미지를 channel_list[s] 만큼의 feature map으로 변환
        self.fromrgb_layers.append(WSConv2d(3, channel_list[s], 1, 1, 0))

        # 해당 해상도 단계의 ConvBlock 추가
        self.prog_blocks.append(DiscriminatorConvBlock(s))

      # 최종 해상도용 fromRGB layer
      self.fromrgb_layers.append(WSConv2d(3,channel_list[0], 1, 1, 0))

      # 최종 Discriminator block: Minibatch -> 3x3 conv -> 4x4 conv -> 1x1 conv -> Sigmoid
      self.prog_blocks.append(nn.Sequential(
          # 배치 표준편차 channel 추가
          MinibatchStd(),
          WSConv2d(channel_list[0]+1, channel_list[0], 3, 1, 1),
          nn.LeakyReLU(0.2),
          WSConv2d(channel_list[0], channel_list[0], 4, 1, 0),
          nn.LeakyReLU(0.2),
          # 최종 실수 scalar값 출력
          WSConv2d(channel_list[0], 1, 1, 1, 0),
          # 0~1 확률값으로 변환
          nn.Sigmoid()
      ))

      # 추가 downsample layer 및 활성화
      self.down = nn.AvgPool2d(2,2)
      self.lrelu = nn.LeakyReLU(0.2)


    # fade-in
    def fade_in(self, alpha, down, cur):
      # down: 이전 해상도의 특징 / cur: 현재 해상도의 특징 / alpha 비율로 보간 -> 점진적 성장 학습 안정화
      return alpha*cur + (1-alpha)*down


    def forward(self, x, alpha):
      # 현재 해상도에서부터 처리 시작: fromRGB -> LeakyReLU
      out = self.lrelu(self.fromrgb_layers[0](x))

      # 현재 단계 = 0인 경우 -> 최종 판별 block으로 이동
      if self.steps==0:
          return self.prog_blocks[-1](out).view(out.size(0), -1)

      # 이전 해상도용 특징: Downsampling -> fromRGB -> LeakyReLU
      down = self.lrelu(self.fromrgb_layers[1](self.down(x)))

      # 현재 해상도 ConvBlock 적용
      out = self.prog_blocks[0](out)

      # fade-in 보간 적용
      out = self.fade_in(alpha, down, out)

      # 남아있는 모든 ConvBlock 연쇄 적용
      for i in range(1, self.steps+1):
        out = self.prog_blocks[i](out)

      # 최종 Discriminator 출력
      return out.view(out.size(0), -1)

## Train

trainer.ipynb 파일 내 코드를 사용하시면 됩니다.

In [ ]:
""" Trainer """
# Generator와 Discriminator 학습 + fade-in 기법 적용
class Trainer():
    def __init__(self, steps, device, train_loader, val_loader, checkpoint_path=None):
      # Progressive 단계 수
      self.steps = steps
      self.device = device
      self.train_loader = train_loader
      self.val_loader = val_loader

      # Generator & Discriminator 모델 -> device
      self.generator = Generator(steps).to(device)
      self.discriminator = Discriminator(steps).to(device)

      # 이진 분류용 loss function -> 진짜 vs 가짜
      self.criterion = nn.BCELoss()

      # Optimizer: Adam
      self.g_opt = Adam(self.generator.parameters(), lr=0.003, betas=(0.0,0.99))
      self.d_opt = Adam(self.discriminator.parameters(), lr=0.001, betas=(0.0,0.99))

      # ---fade-in 제어---
      self.alpha = 0
      # 한 epoch 동안 alpha가 0~1이 되도록 조정
      self.alpha_gap = 1 / (len(train_loader) * 5) # 총 5 epoch동안 전환

      self.final_sample_dir = "/content/drive/MyDrive/ProGAN/Final_images" # 최종 생성 이미지 저장 위치 - 변경 가능
      os.makedirs(self.final_sample_dir, exist_ok=True)

      self.epoch_image_dir = "/content/drive/MyDrive/ProGAN/epoch_images" # 각 epoch 별 이미지 저장 위치 - 변경 가능
      os.makedirs(self.epoch_image_dir, exist_ok=True)

      self.checkpoint_dir = "/content/drive/MyDrive/ProGAN/checkpoints" # checkpoint 저장 위치 - 변경 가능
      os.makedirs(self.checkpoint_dir, exist_ok=True)

      # ---Sampling용 고정 noise---
      # 매 epoch마다 같은 z 입력 -> 생성 결과 변화 비교
      self.test_z = torch.randn(1, 128, 1, 1, device=self.device)
      self.test_z_last = torch.randn(50, 128, 1, 1, device=self.device)

      self.start_epoch = 0
      self.history = {'g_train': [], 'd_train': [], 'g_val': [], 'd_val': []}

      if checkpoint_path is not None:
        self._load_checkpoint(checkpoint_path)

    # Save checkpoint
    def _save_checkpoint(self, epoch):
      path = os.path.join(self.checkpoint_dir, f"checkpoint_epoch{epoch}.pt") # Google Drive 내에 저장하는 것을 추천드립니다.(런타임 끊기는 것을 대비)
      torch.save({
          'epoch': epoch,
          'generator': self.generator.state_dict(),
          'discriminator': self.discriminator.state_dict(),
          'g_opt': self.g_opt.state_dict(),
          'd_opt': self.d_opt.state_dict(),
          'alpha': self.alpha,
          'history': self.history
      }, path)
      print(f"Checkpoint saved: {path}")


    # Load checkpoint
    def _load_checkpoint(self, path):
      checkpoint = torch.load(path)
      self.generator.load_state_dict(checkpoint['generator'])
      self.discriminator.load_state_dict(checkpoint['discriminator'])
      self.g_opt.load_state_dict(checkpoint['g_opt'])
      self.d_opt.load_state_dict(checkpoint['d_opt'])
      self.alpha = checkpoint['alpha']
      self.start_epoch = checkpoint['epoch'] + 1
      self.history = checkpoint['history']
      print(f"Resuming training from epoch {self.start_epoch}")


    # 매 epoch 후 고정 노이즈로 이미지 생성 및 저장
    def save_epoch_image(self, epoch):
      # BatchNorm, Dropout 비활성화
      self.generator.eval()
      with torch.no_grad():
        fake = self.generator(self.test_z)
        save_image(fake[0], f"{self.epoch_image_dir}/epoch_{epoch:03d}.png", normalize=True, value_range=(-1, 1))

    # 마지막 epoch 최종 이미지 생성 및 저장
    def save_last_epoch_image(self, epoch):
      self.generator.eval()
      with torch.no_grad():
        fake = self.generator(self.test_z_last)
        for i in range(fake.size(0)):
          save_image(fake[i], f"{self.final_sample_dir}/epoch_{epoch:03d}_sample_{i:02d}.png", normalize=True, value_range=(-1, 1))


    # 한 epoch동안 Generator와 Discriminator 학습
    def train_epoch(self):
      self.generator.train()
      self.discriminator.train()
      g_loss_avg = 0
      d_loss_avg = 0

      for real in self.train_loader:
        real = real.to(self.device)

        # batch size
        bs = real.size(0)
        real_lbl = torch.full((bs, 1), 0.9, device=self.device)
        fake_lbl = torch.full((bs, 1), 0.1, device=self.device)

        # ---Discriminator Train---
        # 가짜 이미지 생성
        z = torch.randn(bs, 128, 1, 1, device=self.device)
        fake = self.generator(z)

        # 진짜/가짜 판별
        d_fake = self.discriminator(fake.detach(), self.alpha)
        d_real = self.discriminator(real, self.alpha)

        # loss 계산 - 진짜 = 1, 가짜 = 0
        d_loss = self.criterion(d_fake, fake_lbl) + self.criterion(d_real, real_lbl)

        # 역전파 & parameter 업데이트
        self.d_opt.zero_grad()
        d_loss.backward()
        self.d_opt.step()

        # 두 loss의 평균값
        d_loss_avg += d_loss.item()/2

        # ---Generator Train---
        # 새로운 노이즈로 가짜 이미지 생성
        for _ in range(2):
          z = torch.randn(bs, 128, 1, 1, device=self.device)
          fake = self.generator(z)

          # Discriminator가 가짜를 진짜로 판단하도록 유도
          g_loss = self.criterion(self.discriminator(fake, self.alpha), real_lbl)

          # 역전파 & parameter 업데이트
          self.g_opt.zero_grad()
          g_loss.backward()
          self.g_opt.step()
          g_loss_avg += g_loss.item()

        # fade-in 비율 업데이트 - block 간 부드러운 전환
        self.alpha = min(1, self.alpha + self.alpha_gap)

        gc.collect()
        torch.cuda.empty_cache()

      # epcoh 별 평균 loss 반환
      return g_loss_avg/len(self.train_loader), d_loss_avg/len(self.train_loader)


    def valid_epoch(self):
      # 한 epoch 동안 Generator/Discriminator의 validation loss 계산
      self.generator.eval()
      self.discriminator.eval()
      g_loss_avg = 0
      d_loss_avg = 0

      with torch.no_grad():
        for real in self.val_loader:
          real = real.to(self.device)
          bs = real.size(0)
          real_lbl = torch.full((bs, 1), 0.9, device=self.device)
          fake_lbl = torch.full((bs, 1), 0.1, device=self.device)

          # Discriminator 검증
          z = torch.randn(bs, 128, 1, 1, device=self.device)
          fake = self.generator(z)
          d_fake = self.discriminator(fake, self.alpha)
          d_real = self.discriminator(real, self.alpha)
          d_loss_avg += (self.criterion(d_fake, fake_lbl) + self.criterion(d_real, real_lbl)).item() / 2

          # Generator 검증
          z = torch.randn(bs, 128, 1, 1, device=self.device)
          fake = self.generator(z)
          g_loss_avg += self.criterion(self.discriminator(fake, self.alpha), real_lbl).item()

      return g_loss_avg/len(self.val_loader), d_loss_avg/len(self.val_loader)


    # 지정된 epoch 수만큼 학습 및 검증을 수행하고 epoch마다 이미지 저장
    def run(self, epochs):
      for epoch in range(self.start_epoch, epochs):
        # 학습 & validation loss 계산
        g_t, d_t = self.train_epoch()
        g_v, d_v = self.valid_epoch()
        self.history['g_train'].append(g_t)
        self.history['d_train'].append(d_t)
        self.history['g_val'].append(g_v)
        self.history['d_val'].append(d_v)

        # loss log 출력
        print(f"Epoch {epoch}: G_loss {g_t:.4f}/{g_v:.4f}, D_loss {d_t:.4f}/{d_v:.4f}")

        # epoch별 생성 이미지 저장
        self.save_epoch_image(epoch)
        self._save_checkpoint(epoch)

        # 마지막 epoch -> 50개 개별 이미지도 저장
        if epoch == epochs - 1:
          self.save_last_epoch_image(epoch)
          self._save_checkpoint(epoch)

      return self.history

In [ ]:
""" Train """

# 설정
epochs = 40
checkpoint_path = None  # 이어 학습하고 싶은 checkpoint 파일 경로 (없으면 None)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 학습 실행
trainer = Trainer(
    steps=steps,
    device=device,
    train_loader=train_loader,
    val_loader=val_loader,
    checkpoint_path=checkpoint_path # checkpoint 경로
)

history = trainer.run(epochs=epochs)

# 최적 epoch 출력
best_epoch = int(np.argmin(history['g_val']))
print(f"Best epoch: {best_epoch}")

# 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['g_train'], label='G_train')
plt.plot(history['g_val'], label='G_val')
plt.title('Generator Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['d_train'], label='D_train')
plt.plot(history['d_val'], label='D_val')
plt.title('Discriminator Loss')
plt.legend()

plt.show()

# 최종 모델 저장
torch.save(trainer.generator.state_dict(), 'Generator_final.pt')
torch.save(trainer.discriminator.state_dict(), 'Discriminator_final.pt')
print("모델 저장 완료")